In [9]:
from applications.mistral_7b_instruct.utils.graph import graph
from langchain_community.graphs import Neo4jGraph

In [38]:
from applications.mistral_7b_instruct.utils.graph import graph
import itertools

total_query = """
MATCH (n:Disease)
RETURN n.name_fr
"""

results_total = graph.query(total_query)

# Get the count of entries in the results
total_diseases = list(itertools.chain.from_iterable(d.values() for d in results_total))
len(total_diseases)

14275

In [39]:
one_word_query = """
// One word pathology
MATCH (n:Disease)
WHERE not n.name_fr Contains " "
RETURN n.name_fr
"""

results_ow = graph.query(one_word_query)

# Get the count of entries in the results
count = len(results_ow)

print(f"Number of entries: {count}")

results_ow = list(itertools.chain.from_iterable(d.values() for d in results_ow))

Number of entries: 324


In [40]:
results_total = list(set(total_diseases) - set(results_ow))
count = len(results_total)

print(f"Number of entries: {count}")

Number of entries: 13948


In [41]:
two_words_query = """
MATCH (n:Disease)
WHERE size(split(n.name_fr, ' ')) = 2
RETURN n.name_fr
"""

results_tw = graph.query(two_words_query)

# Get the count of entries in the results
count = len(results_tw)

print(f"Number of entries: {count}")

results_tw = list(itertools.chain.from_iterable(d.values() for d in results_tw))
results_total = list(set(results_total) - set(results_tw))
count = len(results_total)

print(f"Number of entries: {count}")

Number of entries: 814
Number of entries: 13136


In [42]:
three_words_query = """
MATCH (n:Disease)
WHERE size(split(n.name_fr, ' ')) = 3
RETURN n.name_fr
"""

results_thw = graph.query(three_words_query)

# Get the count of entries in the results
count = len(results_thw)

print(f"Number of entries: {count}")

results_thw = list(itertools.chain.from_iterable(d.values() for d in results_thw))
results_total = list(set(results_total) - set(results_thw))
count = len(results_total)

print(f"Number of entries: {count}")

Number of entries: 1098
Number of entries: 12039


In [43]:
results_total

['Lésion traumatique du rectum, sans plaie intra-abdominale',
 'Intoxication accidentelle par des pesticides et exposition à ces produits, autres lieux précisés, en pratiquant un sport',
 'Fracture fermée du pubis',
 'Lésions traumatiques des nerfs de plusieurs parties du corps',
 "Intoxication accidentelle par l'alcool et exposition à l'alcool, domicile, en participant à une activité non précisée",
 'Cholestérolose de la vésicule biliaire',
 'Autres pertes de sang foetal',
 'Démence vasculaire sévère, sans précision, avec symptômes essentiellement dépressifs',
 'Autres cyphoses secondaires - Localisation vertébrale non précisée',
 "Échec et rejet d'une greffe de rein",
 'Intoxication accidentelle par le monoxyde de carbone de source non précisée, zone de commerce, en participant à une activité non précisée',
 'Maladie osseuse, sans précision - Avant-bras',
 'Corrosion des organes génito-urinaires internes',
 'Syndrome de sevrage du tabac, sans complication',
 'Autisme atypique en rais

In [46]:
results_kept = results_ow + results_tw + results_thw
len(results_kept)

2236

['Uvéite',
 'bpco',
 'Hémopéricarde',
 'Obésité',
 'Sarcopénie',
 'Autisme',
 'Paraplégie',
 'Tétraplégie',
 'Micatose',
 'stipatose',
 'Lymphangioléiomyomatose',
 'Byssinose',
 'Canitie',
 'Choléra',
 'Botulisme',
 'Balantidiose',
 'Cryptosporidiose',
 'Tularémie',
 'Brucellose',
 'Morve',
 'Spirillose',
 'Streptobacillose',
 'Leptospirose',
 'Pasteurellose',
 'Diphtérie',
 'Coqueluche',
 'Scarlatine',
 'Érysipèle',
 'Gangosa',
 'Typhus',
 'Varicelle',
 'Zona',
 'Variole',
 'Monkeypox',
 'Rougeole',
 'Rubéole',
 'Oreillons',
 'Dermatophytose',
 'Aspergillose',
 'Eumycétome',
 'Actinomycétome',
 'Lobomycose',
 'Rhinosporidiose',
 'Allescheriase',
 'Géotrichose',
 'Pénicilliose',
 'Babésiose',
 'Acanthamoebiose',
 'Naegleriase',
 'Opisthorchiase',
 'Clonorchiase',
 'Dicrocoeliase',
 'Fasciolase',
 'Paragonimiase',
 'Fasciolopsiase',
 'Échinococcose',
 'Cysticercose',
 'Diphyllobothriase',
 'Sparganose',
 'Hyménolépiase',
 'Dracunculose',
 'Onchocercose',
 'Loase',
 'Mansonellose',
 'Ank

In [52]:

autres = [element for element in results_kept if "autre" in element.lower()]
results_kept= list(set(results_kept) - set(autres))


In [53]:
len(results_kept)

2064

In [54]:
import json

with open("maladies.json", "w") as f:
    json_data = json.dumps(results_kept)
    f.write(json_data)

In [58]:
from applications.mistral_7b_instruct.utils.llm import llm_gen

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/amir/.cache/huggingface/token
Login successful


In [84]:
pathologie_template = """
Ecrivez en Français une courte phrase médicale en utilisant la pathologie donnée par l'utilisateur
Ecrivez la phrase comme ce que pourrait dire un médecin dans un compte-rendu médical
Ne répondez que par la phrase à écrire sans formule introductive
Faites en sorte qu'il n'y ait pas d'autre mot relatif à une maladie dans la phrase
\n{pathologie}\n """

from langchain_core.prompts import PromptTemplate

pathologie_prompt = PromptTemplate(
    template=pathologie_template,
    input_variables=["pathologie"]
)

pathologie_chain = pathologie_prompt | llm_gen 

def generate_response(p):
    return pathologie_chain.invoke({"pathologie": p})

In [110]:
results_kept[16]

'Cryptorchidie unilatérale'

In [109]:
generate_response(results_kept[16])

'\nLe patient présente une cryptorchidie gauche.'